In [1]:
import pandas as pd
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split
from surprise import accuracy
from surprise import SVD

In [2]:
peliculas_ratings = pd.read_csv('../PI_1/Datos/Peliculas_Ratings.csv')
usuarios_ratings = pd.read_csv('../PI_1/Datos/Usuarios_Ratings.csv')

# Comenzemos con nuestro sistema de ML

Primero extraemos peliculas y titulos

In [5]:

peliculas = peliculas_ratings[['id','title']]
peliculas = peliculas.rename(columns={'id':'movieId'})
print('Peliculas\n-------------------------------------------------------------')
print(peliculas.head(3))
print('-------------------------------------------------------------')

Peliculas
-------------------------------------------------------------
  movieId                 title
0     as1   the grand seduction
1     as2  take care good night
2     as3  secrets of deception
-------------------------------------------------------------


In [7]:
#Usuarios que dieron una calificacion
print('Usuarios\n-------------------------------------------------------------')
usuarios = usuarios_ratings.drop(columns=['timestamp'])
print(usuarios.head(3))
print('-------------------------------------------------------------')

Usuarios
-------------------------------------------------------------
   userId  score movieId
0       1    1.0   as680
1       1    4.5  ns2186
2       1    5.0  hs2381
-------------------------------------------------------------


Agrupamos peliculas por cantidad de calificaciones, lo que tomaremos como vistas

In [8]:
print('Peliculas por popularidad\n-------------------------------------------------------------')
#Agrupamos peliculas por cantidad de calificaciones y reseteamos el indice
vistas = usuarios[['movieId','score']].groupby(['movieId']).count().reset_index()
#Renombramos la cantidad de calificaciones como vistas
vistas.rename(columns = {'score':'vistas'}, inplace = (True))
#Ordenamos
vistas.sort_values(['vistas'], ascending = False, inplace = True)
#Les agregamos el titulo de las peliculas
vistas = pd.merge(vistas,peliculas,on='movieId')
print(vistas.head())

Peliculas por popularidad
-------------------------------------------------------------
  movieId  vistas                             title
0  as4197     576                 from other worlds
1  as9065     560                  the organization
2  ns7279     558  leapfrog: sing-along, read-along
3   as848     558                         superbook
4  hs2360     556                          dietland
-------------------------------------------------------------


In [14]:
int(vistas.vistas.mean())

479

Detectamos peliculas con pocas vistas

In [15]:
print('Vistas máximas:',vistas.vistas.max())
print('Vistas máximas:',vistas.vistas.min())
print('Promedio de vistas:',int(vistas.vistas.mean()), '. Este determinará hasta cuando una pelicula se clasifica con pocas vistas')
umbral = int(vistas.vistas.mean())
mascara_pocos_vistos = vistas[vistas['vistas']<umbral]

Vistas máximas: 576
Vistas máximas: 396
Promedio de vistas: 479 . Este determinará hasta cuando una pelicula se clasifica con pocas vistas


In [16]:
#Extraemos el indice de las peliuclas que se clasifican con pocos vistos
peliculas_pocos_vistos = mascara_pocos_vistos.index.values

In [19]:
#Invocamos Reader class de la libreria surplrise, que se encargara de analizar archivos que contienen calificaciones
reader = Reader()

In [20]:
#Usamos el metodo de clase load_from_df de surprise para cargar un conjunto de datos de pandas, a la clase Reader()
data = Dataset.load_from_df(usuarios[['userId','movieId','score']], reader)

In [ ]:
# Separamos nuestros datos
trainset, testset = train_test_split(data, test_size=.25)
# Usaremos un modelo de Singular Value Decomposition
model = SVD()
# Entrenamos el modelo
model.fit(trainset)
# Predecimos
predictions = model.test(testset)

In [ ]:
#Creamos una lista para validar que el usuario existe en nuestra funcion
users = usuarios_ratings.userId.unique()
users = list(users)

In [ ]:
def recomendacion_peliculas(id_usuario):

    if type(id_usuario) != int:
        return {'Por favor ingrese un numero entero, entre los valores (1 - 124380)'}
    #Verificamos que exita el usuario ingresado
    if id_usuario not in users:
        return {'El usuario ingresado no existe, por favor ingrese otro id, entre los valores (1 - 124380)'}

    recomendaciones_usuario = peliculas
    #Definimos las peliculas que ya vio en uan nueva variable
    usuario_vistas = usuarios[usuarios['userId'] == id_usuario]
    #Quitamos las peliculas con pocas vistas de nuestros datos
    for i in peliculas_pocos_vistos:
        if i in (recomendaciones_usuario.index.values):
            recomendaciones_usuario.drop(i,inplace=True)
    #Agregamos las recomendaciones que predijo nuestro modelo
    recomendaciones_usuario['Estimate_Score'] = recomendaciones_usuario['movieId'].apply(lambda x: model.predict(id_usuario, x).est)
    #Ordenamos segun la valoracion del usuario predicha para la pelicula recomendada
    recomendaciones_usuario = recomendaciones_usuario.sort_values('Estimate_Score', ascending=False)
    return print('Las siguientes peliculas son las recomendadas para el usuario id', id_usuario,'\n-------------------------------------------------------------\n',recomendaciones_usuario.head(10),'\n-------------------------------------------------------------\nEl rmse es de',accuracy.rmse(predictions))

In [ ]:
recomendacion_peliculas()